We can ignore the cell above

In [ ]:
import base64
import cherrypy
from os.path import isfile, join
from os import listdir


class AddModel:
    exposed = True  # Needed for exposing the Web Services

    def POST(self, **query):
        if len(query) != 2:
            raise cherrypy.HTTPError(400, 'Wrong query')

#         if type(query.get('model')) != 'bytes':
#             raise cherrypy.HTTPError(400, 'Model type is not bytes')

        if not query.get('name').endswith('.tflite'):
            raise cherrypy.HTTPError(400, 'The name extention should be tflite')
        model = query.get('model')
        model_name = query.get('name')
        decoded_model = base64.b64decode(model)
        path = './model/' + str(model_name)
        with open(path, 'wb') as f:
            f.write(decoded_model)


class ListModels:
    exposed = True  # Needed for exposing the Web Services

    def GET():
        models_path = './model'
        onlyFiles = [f for f in listdir(models_path) if isfile(join(models_path, f))]
        for i in onlyFiles:
            if i.endswith('.tflite'):
                print(i)


class Predict:
    exposed = True  # Needed for exposing the Web Services
    def PUT(self,*path):
        model_name=path[0]
        tthres=path[1]
        hthres=path[2]
        models_path='./model'
        with open(models_path+'/'+model_name, 'rb') as fp:
            model = fp.read()#now we have the model
            

if __name__ == '__main__':
    # conf probably needs modification
    conf = {
        '/': {
            'request.dispatch': cherrypy.dispatch.MethodDispatcher(),
            'tools.sessions.on': True,
        }
    }
    cherrypy.tree.mount(AddModel(), '/add', conf)
    cherrypy.tree.mount(ListModels, '/list', conf)
    # cherrypy.tree.mount(Predict, '/predict', conf)

    # To start cherrypy engine
    cherrypy.engine.start()
    cherrypy.engine.block()